![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

# Set up
## Import Libraries

In [54]:
import pandas as pd
import numpy as np
# Read csv
bank_marketing = pd.read_csv('bank_marketing.csv')
print(bank_marketing.info())
bank_marketing.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day             41188 non-null  int64  
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        41188 non-null  object 
 16  emp_var_rate    41188 non-null  float64
 17  cons_price_idx  41188 non-null 

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,13,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,19,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,23,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,27,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,3,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


## Split bank_marketing.csv into 3 DataFrames
### client

In [55]:
# define columns
client_columns = [
    'client_id', 'age', 'job', 'marital', 
    'education', 'credit_default', 'housing', 'loan'
]
client_col_rename = {
    'client_id':'id'
}
client_na = {
    'education':'unknown','credit_default':'unknown','housing':'unknown','loan':'unknown'
}
# DataFrame
client = pd.read_csv('bank_marketing.csv', usecols=client_columns,na_values=client_na)
client = client.rename(columns=client_col_rename)
# Cleaning
client['education'] = client['education'].str.replace('.','_')
client['job'] = client['job'].str.replace('.','')

print(client.info())
client.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       39457 non-null  object
 5   credit_default  32591 non-null  object
 6   housing         40198 non-null  object
 7   loan            40198 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.5+ MB
None


,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,NaN,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


### campaign

In [56]:
# define columns
campaign_columns = [
    'client_id', 'campaign', 'duration', 
    'pdays', 'previous', 'poutcome', 
    'y', 'day', 'month'
]
campaign_col_rename = {
    'campaign':'number_contacts', 'duration':'contact_duration', 'previous':'previous_campaign_contacts',
    'poutcome':'previous_outcome', 'y':'campaign_outcome'
}
campaign_na = {
    'poutcome':'nonexistent'
}
# DataFrame
campaign = pd.read_csv('bank_marketing.csv', usecols=campaign_columns,na_values=campaign_na)
campaign = campaign.rename(columns=campaign_col_rename)
#cleaning
campaign['previous_outcome'] = campaign['previous_outcome'].str.replace('success', '1')
campaign['previous_outcome'] = campaign['previous_outcome'].str.replace('failure', '0')
campaign['campaign_outcome'] = campaign['campaign_outcome'].str.replace('yes', '1')
campaign['campaign_outcome'] = campaign['campaign_outcome'].str.replace('no', '0')
#adding column
campaign['campaign_id'] = np.arange(0,campaign.shape[0]) + 1
campaign['last_contact_date'] = '2022-' + pd.to_datetime(campaign['month'], format='%b').dt.strftime('%m') \
    + '-' + campaign['day'].astype(str)

# reording and removal of redundant columns
campaign = campaign[['campaign_id', 'client_id','contact_duration', 
                     'number_contacts', 'pdays', 'previous_campaign_contacts', 
                     'previous_outcome', 'campaign_outcome', 'last_contact_date']]

print(campaign.info())
campaign.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   campaign_id                 41188 non-null  int64 
 1   client_id                   41188 non-null  int64 
 2   contact_duration            41188 non-null  int64 
 3   number_contacts             41188 non-null  int64 
 4   pdays                       41188 non-null  int64 
 5   previous_campaign_contacts  41188 non-null  int64 
 6   previous_outcome            5625 non-null   object
 7   campaign_outcome            41188 non-null  object
 8   last_contact_date           41188 non-null  object
dtypes: int64(6), object(3)
memory usage: 2.8+ MB
None


,campaign_id,client_id,contact_duration,number_contacts,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,1,0,261,1,999,0,NaN,0,2022-05-13
1,2,1,149,1,999,0,NaN,0,2022-05-19
2,3,2,226,1,999,0,NaN,0,2022-05-23
3,4,3,151,1,999,0,NaN,0,2022-05-27
4,5,4,307,1,999,0,NaN,0,2022-05-3


### economics

In [57]:
# Define columns
economics_columns = [
    'client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed'
]
economics_col_rename = {
    'euribor3m':'euribor_three_months', 'nr_employed':'number_employed'
}
# DataFrame
economics = pd.read_csv('bank_marketing.csv', usecols=economics_columns)
economics = economics.rename(columns=economics_col_rename)

print(economics.info())
economics.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   emp_var_rate          41188 non-null  float64
 2   cons_price_idx        41188 non-null  float64
 3   euribor_three_months  41188 non-null  float64
 4   number_employed       41188 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 1.6 MB
None


,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


## DataFrames saved to csv

In [58]:
# Save dataframes to CSV
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)

## Variables using SQL defining the creation of tables.

In [59]:
client_table = '''
CREATE TABLE client (
    id serial PRIMARY KEY,
    age integer,
    job text,
    marital text,
    education text,
    credit_default boolean,
    housing boolean,
    loan boolean
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
'''
campaign_table = '''
CREATE TABLE campaign (
    campaign_id serial PRIMARY KEY,
    client_id serial REFERENCES client (id),
    number_contacts integer,
    contact_duration integer,
    pdays integer,
    previous_campaign_contacts integer,
    previous_outcome boolean,
    campaign_outcome boolean,
    last_contact_date date
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
'''
economics_table = '''
CREATE TABLE economics (
    client_id serial REFERENCES client (id),
    emp_var_rate float,
    cons_price_idx float,
    euribor_three_months float,
    number_employed float
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
'''